In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [5]:
pwd

'C:\\Users\\jaklo'

In [6]:
#Capturing file path
train_data = pd.read_csv('C://Users//jaklo/iowa/train.csv', index_col = 'Id')
test_data = pd.read_csv('C://Users//jaklo/iowa/test.csv', index_col='Id')

#Remove missing values. Separate target from predictors
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = train_data.SalePrice
train_data.drop(['SalePrice'], axis=1, inplace=True)

train_data.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000


In [7]:
#Breaking data in train and validation
X_train_full, X_valid_full, y_train, y_valid = train_test_split(train_data, y, train_size=0.8, test_size=0.2,
                                                               random_state = 0)

#Selecting numerical columns and categorical columns with low cardinality
low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique()<10 and
                       X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname]. dtype in ['int64', 'float64']]

#Keeping selected columns only
my_cols=low_cardinality_cols + numerical_cols
X_train=X_train_full[my_cols].copy()
X_valid=X_valid_full[my_cols].copy()
X_test=test_data[my_cols].copy()

In [8]:
#One hot encode the data
X_train=pd.get_dummies(X_train)
X_valid=pd.get_dummies(X_valid)
X_test=pd.get_dummies(X_test)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [9]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error as mae

ModuleNotFoundError: No module named 'xgboost'

In [10]:
#Define and fit ensemble model with Extreme Gradient Regressor
my_model= XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train,
            early_stopping_rounds = 5,
            eval_set =[(X_valid, y_valid)], verbose=False)

[17:37:41] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.05, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)

In [13]:
#Get predictions
preds = my_model.predict(X_valid)

#Calculate MAE
MAE = mae(preds, y_valid)

print("Mean Absolute Error:", MAE)

Mean Absolute Error: 16751.568760702055


In [14]:
#Get predictions on test data
preds_test = my_model.predict(X_test)

print(my_model.predict(X_test.head(1460)))

[117881.586 164481.72  183055.23  ... 172999.31  120539.93  241051.1  ]


In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
sns.distplot (a=(my_model.predict(X_test.head(1460))), kde=False)

NameError: name 'my_model' is not defined